<a href="https://colab.research.google.com/github/shu65/pytorch-cuda-time-measurement/blob/main/Pytorch_GPU_Time_Measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!nvidia-smi

Sat Mar 20 05:36:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   2914MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
import torch
input_batch_cpu = torch.randn(128, 3, 224, 224)
input_batch_gpu = input_batch_cpu.clone().detach().to('cuda')

In [18]:
import torch
model_cpu = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
model_gpu = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
model_gpu.to('cuda')

model_cpu.eval()
model_gpu.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
import time

start = time.time()
with torch.no_grad():
  out = model_cpu(input_batch_cpu)
elapsed_time = time.time() - start

print(elapsed_time)

18.83014965057373


In [20]:
import time

start = time.time()
with torch.no_grad():
  out = model_gpu(input_batch_gpu)
elapsed_time = time.time() - start

print(elapsed_time, 'sec.')

0.011448860168457031 sec.


In [21]:
import time

torch.cuda.synchronize()
start = time.time()
with torch.no_grad():
  out = model_gpu(input_batch_gpu)
torch.cuda.synchronize()
elapsed_time = time.time() - start

print(elapsed_time, 'sec.')

0.322756290435791 sec.


In [22]:
import time

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
with torch.no_grad():
  out = model_gpu(input_batch_gpu)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)

print(elapsed_time / 1000, 'sec.')

0.32002630615234373 sec.
